In [34]:
import pandas as pd
import numpy as np
import random

In [28]:
class quadtree_node():
    def __init__(self, x: float, y: float ):
        self.x = x
        self.y = y

        self.branches = {
            'nw': None,
            'ne': None,
            'se': None,
            'sw': None
        }
    
    def get_quadrant(self, x, y):
        if y >= self.y:
            point = 'n'
        else:
            point = 's'
        if x >= self.x:
            point += 'e'
        else:
            point += 'w'
        return point

# =============================================================================

    def add_element(self, x: float, y: float):
        point = self.get_quadrant(x, y)
        if self.branches[point] is None:
            self.branches[point] = quadtree_node(x, y)
        else:
            self.branches[point].add_element(x, y)

# =============================================================================

    def search_element(x,y):
        if (self.x == x) and (self.y == y):
            return True
        else:
            point = self.get_quadrant(x,y)
            if self.branches[point] is not None:
                return self.branches[point].search_element(x,y)
            else:
                return False

# =============================================================================

    def print(self, lv=0):
        print(f"[{self.x}, {self.y}]", end='')
        flag = False
        for x in self.branches:
            if self.branches[x] is not None:
                flag = True
                break
        if not flag:
            print('')
            return
        else:
            print(':')
            for x in self.branches:
                blank = ''
                for i in range(lv+1):
                    blank += '  '
                print(f"{blank}{x}:", end='')
                
                if self.branches[x] is None:
                    print("-null-")
                else:
                    self.branches[x].print(lv+1)



In [29]:
class test():
    def __init__(self):
        self.x = 5
        self.y = 10

        self.vals = {'x': 5,
        'y': 10}

    def add(self, var, val):
        self.vals[var] += val
    
    def add_x(self, val):
        self.add('x', val)
    
    def add_y(self, val):
        self.add('y', val)


In [30]:
t = test()
t.add_x(3)
print(t.vals['x'])
print(t.vals['y'])

8
10


In [39]:
q_tree = quadtree_node(5, 7)

In [40]:
q_tree.add_element(4, 6)

In [46]:
for i in range(100):
    x = random.random()
    y = random.random()
    print(f"({x}, {y})")
    q_tree.add_element(x, y)


(0.3205175290883189, 0.810278033834329)
(0.38115190051272263, 0.30524631194056806)
(0.9648435707706057, 0.7709245334523279)
(0.13262396280073818, 0.38403028117390847)
(0.6143512858674657, 0.9529649142619153)
(0.18065850675956696, 0.8553318742624806)
(0.17048937055886726, 0.9855683080909855)
(0.17930363839063979, 0.6501164715031198)
(0.8487408907818733, 0.22907128528085008)
(0.875308657911485, 0.13078657468742494)
(0.682152904061594, 0.12864702903216985)
(0.5256239310564944, 0.12851367001297553)
(0.8019647338632606, 0.08732538931203915)
(0.7854759133127375, 0.673527557750993)
(0.9899010774147073, 0.6954680982832326)
(0.19961253044157545, 0.7615707046794442)
(0.6105276121744296, 0.17091183058509485)
(0.7173365091464269, 0.19486840724525223)
(0.7010767859541919, 0.11591046814192163)
(0.9126278847742311, 0.8923680162239361)
(0.264124760586805, 0.20825455449906172)
(0.7212830268126027, 0.45615829685975817)
(0.17800585715467032, 0.1842551407934574)
(0.5088840659331733, 0.5759836283818209)
(0

In [47]:
q_tree.print()

[5, 7]:
  nw:[3, 42]:
    nw:[1, 72]
    ne:-None-
    se:-None-
    sw:-None-
  ne:[17, 62]:
    nw:[5, 88]:
      nw:-None-
      ne:-None-
      se:[14, 74]
      sw:-None-
    ne:[30, 91]:
      nw:[24, 97]:
        nw:-None-
        ne:-None-
        se:[29, 93]
        sw:-None-
      ne:[36, 94]:
        nw:-None-
        ne:[92, 99]
        se:[94, 91]
        sw:-None-
      se:[69, 81]:
        nw:[50, 83]:
          nw:[46, 83]
          ne:[50, 84]
          se:-None-
          sw:-None-
        ne:[96, 83]
        se:[82, 80]:
          nw:-None-
          ne:-None-
          se:[98, 65]:
            nw:[93, 77]
            ne:[99, 65]
            se:-None-
            sw:[91, 62]
          sw:[76, 74]:
            nw:-None-
            ne:-None-
            se:[77, 65]
            sw:-None-
        sw:[50, 64]:
          nw:[48, 76]:
            nw:[43, 77]
            ne:-None-
            se:-None-
            sw:[41, 74]:
              nw:[40, 74]:
                nw:[